In [13]:
import numpy as np
import scipy.linalg as la
import asymmetric as asym
import quantum as qm
%matplotlib inline
import matplotlib.pyplot as plt
import krypy

In [2]:
def get_vec_Haar(N):
    vec = np.random.normal(size=N) + 1j * np.random.normal(size=N)
    norm = la.norm(vec)
    return vec/norm

In [68]:
######################################################
# Presets for problem
######################################################
L = 5
H = asym.dense_H(L)
maxiter = L^2
tol = 10e-10

In [39]:
######################################################
# Setup for a specific run
######################################################
t = 1
v = get_vec_Haar(2**L)
ret = krypy.utils.Arnoldi(-1j*H*t, np.array([v]).T)

In [63]:
######################################################
# At time t, try iter iterations until v(t) converges?
# Then use the V_m you've found to make U(t), U^dag(t)
# Once you've done this once, use the same m
######################################################
for iter in 
ret.advance()

In [66]:
stuff = ret.get()
print(np.shape(stuff[0]), np.shape(stuff[1]))
print()
# for thing in stuff:
#     print(thing)
#     print()

(32, 6) (6, 5)



In [67]:
print(qm.chop(stuff[0][:,0]-v))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
